### Step 1: Data Preparation ###

Minimal data preparation is performed in order to maintain the syntax between Quirk and QuirKit.

The following URL will create the circuit in Quirk for full reproducability:
https://algassert.com/quirk#circuit={%22cols%22:[[%22H%22,1,%22H%22],[%22Swap%22,%22Swap%22,%22H%22,1,%22H%22],[1,%22%E2%80%A2%22,%22X%22],[%22Y%22,1,1,%22H%22],[1,%22Y%22],[1,%22%E2%80%A2%22,%22%E2%80%A2%22,%22X%22],[1,1,1,1,%22Y%22],[%22H%22]],%22init%22:[0,1]}

In [51]:
qjson = """
{
  "cols": [
    [
      "•",
      "X"
    ],
    [
      "X",
      "•"
    ],
    [
      "•",
      "X"
    ]
  ],
  "init": [
    1
  ]
}
"""

### Importing the JSON Data ###

This section is to confirm to the transfer of the exported JSON to a Python JSON object; it is not necessary for the final version of the tool and exists as a troubleshooting measure if needed.

In [52]:
import json
from types import SimpleNamespace

qdata = json.loads(qjson, object_hook=lambda d: SimpleNamespace(**d))
print(qjson)
print(qdata)


{
  "cols": [
    [
      "•",
      "X"
    ],
    [
      "X",
      "•"
    ],
    [
      "•",
      "X"
    ]
  ],
  "init": [
    1
  ]
}

namespace(cols=[['•', 'X'], ['X', '•'], ['•', 'X']], init=[1])



### Conversion to Qiskit ###

In [53]:
import numpy as np
import math as m
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector
import pyperclip

def QuirKitUp(qdata, showOutput=True, copyToClip=True):
    #Baseline: All outputs will require the following:
    baseline = """#Generated by QuirKit
from qiskit import QuantumCircuit, assemble, transpile
from qiskit.providers.aer import QasmSimulator
from qiskit.quantum_info import Statevector
sim = QasmSimulator()"""

    #First, we have to determine how many qubits the data represents.
    #This is not given in the data; instead we must iterate over the data
    # and find which row has the most elements. Since a single operation
    # on a qubit that is the last in a column will generate a row that is
    # the number of qubits in length of the circuit, we are guaranteed
    # to find the correct number.
    qlen = 0
    for c in qdata.cols:
        if c.__len__() > qlen:
            qlen = c.__len__()
    if showOutput == True:
        print("Qubit Size: ", qlen)
    qrange = list(range(qlen))

    #The initial states of the qubits are determined by the init object,
    # but only for the states which are non-trailing-zeros. Defaults are
    # added here for readability in the final circuit code.
    initializers = """#Qubit State Initialization\n"""
    if (hasattr(qdata, 'init')):
        for i, ini in enumerate(qdata.init):
            if ini == 0:
                initializers += """circuit.initialize(Statevector.from_label('0'), {})\n""".format(i)
            if ini == 1:
                initializers += """circuit.initialize(Statevector.from_label('1'), {})\n""".format(i)
            if ini == "+":
                initializers += """circuit.initialize(Statevector.from_label('+'), {})\n""".format(i)
            if ini == "-":
                initializers += """circuit.initialize(Statevector.from_label('-'), {})\n""".format(i)
            if ini == "i":
                initializers += """circuit.initialize(Statevector.from_label('r'), {})\n""".format(i)
            if ini == "-1":
                initializers += """circuit.initialize(Statevector.from_label('l'), {})\n""".format(i)
        for i, q in enumerate(range(qdata.init.__len__(), qlen)):
            initializers += """circuit.initialize(Statevector.from_label('0'), {})\n""".format(i+qdata.init.__len__())
    else:
        initializers = """#No Initialization Data Received: All Qubits Set to |0>"""

    # Create the circuit with qlen
    circuit_init = """circuit = QuantumCircuit({}, {})\n""".format(qlen, qlen)
    
    # Create the circuit compilation
    compiler = """compiled_circuit = transpile(circuit, sim)"""
    counter = """job = sim.run(compiled_circuit, shots=1024)
result = job.result()
counts = result.get_counts(compiled_circuit)\n"""

    # Generate all the gates and operations.
    gates = """#Quantum Gates\n"""
    for c in qdata.cols:
        qcontrol = -1
        qxcontrol = -1
        swaps = []
        #Row Level Operations
        for i, el in enumerate(c):
            #Column Level Operations
            #First, handle the simple gates
            if el == "H":
                gates += """circuit.h({})""".format(i) + "\n"
            if el == "Y":
                gates += """circuit.y({})""".format(i) + "\n"
            #Second, handle the complex gates
            if el == "•":
                qcontrol = i
            if el == "X":
                qxcontrol = i
            if el == "Swap":
                swaps.append(i)
        if qcontrol != -1 and qxcontrol != -1:
            gates += """circuit.cx({},{})""".format(qcontrol, qxcontrol) + "\n"
        if swaps.__len__() == 2:
            gates += """circuit.swap({},{})""".format(swaps[0], swaps[1]) + "\n"

    #As Quirk measures all qubits at the end of a circuit, we add a
    # measurement line for each qubit.
    measure = """#Measurement & Results
circuit.measure({}, {})""".format(qrange, qrange)

    #The draw instruction is the final addition.
    draw = """#Output
print(counts)
circuit.draw()"""


    #Assemble Output
    #-start with baseline
    final = baseline 
    final += "\n"
    final += circuit_init
    final += "\n"
    final += initializers
    final += "\n"
    final += gates
    final += "\n"
    final += measure
    final += "\n"
    final += compiler
    final += "\n"
    final += counter
    final += "\n"
    final += draw
    #-build circuit

    if showOutput == True:
        print("Qiskit Code:")
        print(final)
    if copyToClip == True:
        pyperclip.copy(final)

### Functional Test ###

In [54]:
QuirKitUp(qdata)

Qubit Size:  2
Qiskit Code:
#Generated by QuirKit
from qiskit import QuantumCircuit, assemble, transpile
from qiskit.providers.aer import QasmSimulator
from qiskit.quantum_info import Statevector
sim = QasmSimulator()
circuit = QuantumCircuit(2, 2)

#Qubit State Initialization
circuit.initialize(Statevector.from_label('1'), 0)
circuit.initialize(Statevector.from_label('0'), 1)

#Quantum Gates
circuit.cx(0,1)
circuit.cx(1,0)
circuit.cx(0,1)

#Measurement & Results
circuit.measure([0, 1], [0, 1])
compiled_circuit = transpile(circuit, sim)
job = sim.run(compiled_circuit, shots=1024)
result = job.result()
counts = result.get_counts(compiled_circuit)

#Output
print(counts)
circuit.draw()
